In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd
import os

In [3]:
year = 2021
month = 2

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/pop/envs/mlops/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pop/envs/mlops/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
filepath = f"../../dataset/fhv_tripdata_{year:04d}-{month:02d}.parquet"
if not os.path.exists(filepath):
        print('Downloading file...')
        fname = os.path.basename(filepath)
        os.system(f'wget https://nyc-tlc.s3.amazonaws.com/trip+data/{fname} -P ../../dataset/')


In [7]:
df = read_data(filepath)

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

# Q1

In [10]:
mean_pred_duration = y_pred.mean()
mean_pred_duration

16.191691679979066

# Q2

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [11]:
df.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration'],
      dtype='object')

In [19]:
def save_results(df, y_pred, output_file):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    # df_result['pickup_datetime'] = df['pickup_datetime']
    # df_result['PUlocationID'] = df['PUlocationID']
    # df_result['DOlocationID'] = df['DOlocationID']
    # df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    # df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']

    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
        )

In [20]:
output_file = f'output/{os.path.basename(filepath)}'
save_results(df, y_pred, output_file)

In [21]:
!ls -lh ./output

total 19M
-rw-rw-r-- 1 pop pop 19M Jun 21 21:41 fhv_tripdata_2021-02.parquet


# Q3

In [22]:
# jupyter nbconvert --to script starter.ipynb